In [1]:
import pandas as pd
import os
from google import genai
from docx import Document
import re

In [2]:
with open('C:/cred_google_gemini.txt', 'r') as file:
    llave = file.read()

In [3]:
def limpiar_texto(x):
    if isinstance(x, str):
        x = re.sub(r"A\\n", "", x)     # eliminar "A\n"
        x = re.sub(r"\\n", " ", x)     # reemplazar "\n" por espacio
        x = re.sub(r"\\t", " ", x)     # reemplazar "\t" por espacio
        x = re.sub(r"•", "", x)        # eliminar viñetas
        x = re.sub(r"\s+", " ", x)  
        #Eliminar "-"
        x= re.sub(r"-", "", x)
        x= re.sub(r"●", "", x)
        return x.strip()
    return x

In [5]:
##Leer matriz de consolidación
matriz=pd.read_excel("H:/Mi unidad/cuestionarios/visiones_sectorizados_pgd.xlsx")
matriz.columns=matriz.columns.str.lower()  
#Renombrar segunda columna como "otro"
matriz.rename(columns={matriz.columns[1]: 'otro',
                           matriz.columns[2]: 'corto_plazo_1',
    matriz.columns[3]: 'corto_plazo_2',
    matriz.columns[4]: 'mediano_plazo',
    matriz.columns[5]: 'largo_plazo'
    
    }, inplace=True)
matriz['dependencia'] = matriz.apply(
    lambda row: row["otro"]
    if row['dependencia'] == 'Otro' and pd.notna(row["otro"])
    else row['dependencia'],
    axis=1
)
matriz["dependencia"] = matriz["dependencia"].str.title()


#Eliminar columna otro
del matriz['otro']

#Eliminar espacios en blanco en todas las columnas
matriz = matriz.map(limpiar_texto)


matriz

,dependencia,corto_plazo_1,corto_plazo_2,mediano_plazo,largo_plazo,fuente
0,"Sectorizados Educación, Ciencia, Tecnología E ...",Establecer las bases de una política pública d...,Establecer a la Ciudad de México como un model...,"Construir un sistema deportivo sólido, incluye...","Asegurarcondicionesestructurales,normativasyde...",NaN
1,"Sectorizados Educación, Ciencia, Tecnología E ...",OBJETIVO GENERAL Contribuir en educación de ca...,OBJETIVO GENERAL Contribuir en educación de ca...,OBJETIVO GENERAL Contribuir en educación de ca...,OBJETIVO GENERAL Contribuir en educación de ca...,NaN
2,"Sectorizados Educación, Ciencia, Tecnología E ...",Objetivo general: Garantizar condiciones mínim...,Objetivo general: Construir una base sólida qu...,Objetivo general: Consolidar al IEMS como el m...,Objetivo general: Consolidar un modelo educati...,NaN
3,"Sectorizados Educación, Ciencia, Tecnología E ...",Objetivo general Consolidar las bases académic...,Objetivo general Consolidar un modelo educativ...,Objetivo general Transformar estructuralmente ...,Objetivo general Consolidar un ecosistema educ...,NaN
4,"Sectorizados Educación, Ciencia, Tecnología E ...",Objetivo General: Profesionalizar al servicio ...,Objetivo general Consolidar la profesionalizac...,Objetivo General Transformar estructuralmente ...,Objetivo General Establecer a la Ciudad de Méx...,NaN
5,"Sectorizados Educación, Ciencia, Tecnología E ...","Objetivo general. Para 2027, consolidar plenam...","Objetivo general. Para 2030, garantizar que to...","Objetivo general. Para el año 2040, consolidar...","Objetivo general. Para el año 2050, el Subsist...",NaN


In [6]:
matriz["dependencia"].unique()

array(['Sectorizados Educación, Ciencia, Tecnología E Innovación'],
      dtype=object)

In [7]:
#Pasar a long
matriz_long = pd.melt(matriz, id_vars=['dependencia'], 
                      value_vars=['corto_plazo_1', 'corto_plazo_2', 'mediano_plazo', 'largo_plazo'],
                      var_name='plazo', value_name='vision')
#Renombrar corto_plazo_1 y corto_plazo_2 como corto_plazo
matriz_long['plazo'] = matriz_long['plazo'].replace({'corto_plazo_1': 'corto_plazo',
                                                    'corto_plazo_2': 'corto_plazo'})
matriz_long = matriz_long.groupby(['dependencia', 'plazo'], as_index=False).agg({
    'vision': lambda x: '. '.join(x).strip()
})
matriz_long

,dependencia,plazo,vision
0,"Sectorizados Educación, Ciencia, Tecnología E ...",corto_plazo,Establecer las bases de una política pública d...
1,"Sectorizados Educación, Ciencia, Tecnología E ...",largo_plazo,"Asegurarcondicionesestructurales,normativasyde..."
2,"Sectorizados Educación, Ciencia, Tecnología E ...",mediano_plazo,"Construir un sistema deportivo sólido, incluye..."


In [8]:
# 2) Función que arma el “documento” por grupo
def documento_por_grupo(g, salto="\n\n"):
    corto = salto.join(
        f"Corto plazo: {t.strip()}"
        for t in g.loc[g.plazo.str.contains("largo_plazo", na=False), "vision"]
    )
    mediano  = salto.join(
        f"Objetivo: {t.strip()}"
        for t in g.loc[g.plazo.str.contains("largo_plazo", na=False), "vision"]
    )
    largo  = salto.join(
        f"Estrategia: {t.strip()}"
        for t in g.loc[g.plazo.str.contains("largo_plazo", na=False), "vision"]
    )
    return corto, mediano, largo

In [15]:
# 4. Crear carpeta de salida si no existe
if not os.path.exists("visiones_dependencias"):
    os.makedirs("visiones_dependencias")

In [16]:
grupos = (
    matriz_long.groupby(["dependencia"], dropna=False)
          .apply(documento_por_grupo)
)

print(f"Se generarán {len(grupos)} documentos Word")
grupos

Se generarán 1 documentos Word


C:\Users\claud\AppData\Local\Temp\ipykernel_26328\416768345.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(documento_por_grupo)


dependencia
Sectorizados Educación, Ciencia, Tecnología E Innovación    (Corto plazo: Asegurarcondicionesestructurales...
dtype: object

In [17]:
client=genai.Client(api_key=llave)

In [19]:
for clave, (corto, mediano, largo) in grupos.items():
    dependencia = clave

    # --- Prompt con prefijos ya incluidos -----------------------------------
    prompt = (
        "Redacta un documento sintético de corrido que genere la visión general a futuro utilizando la siguiente información:\n\n"
        f"{corto}\n\n{mediano}\n\n{largo}"
        "Dale coherencia. No repitas la información, no uses viñetas ni listas y no inventes información.\n"
        "El documento debe señalar a corto plazo (2030), mediano plazo (2040) y largo plazo (2050).\n"
        "El documento debe ser de no más de dos cuartillas.\n"
        "El documento considera visiones desde el punto de vista del deporte, educación en materia de salud, de educación media superior, educación de funcionarios en la administración pública y de becas para estudiantes. Considera todas\n"
        "El documento debe ir de corrido, sin viñetas ni listas, y debe tener coherencia y cohesión.\n"
    )

    resp = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[prompt]
    )
    resumen = resp.candidates[0].content.parts[0].text.strip()


    # --- Crear DOCX ----------------------------------------------------------
    doc = Document()
    doc.add_heading(f"{dependencia}", level=1)
    #doc.add_paragraph(f"Eje: {eje}")
    #doc.add_paragraph(f"Documento fuente: {documento}")
    doc.add_heading("Documento", level=2)
    doc.add_paragraph(resumen)

    # Guardar
    nombre = f"{dependencia[:30]}.docx".replace(" ", "_")
    #nombre = f"{linea[:30]}_{tema[:20]}.docx".replace(" ", "_")
    ruta = os.path.join("visiones_dependencias", nombre)
    doc.save(ruta)
    print("✓ Guardado:", ruta)

✓ Guardado: visiones_dependencias\Sectorizados_Educación,_Cienci.docx
